In [88]:
# Initializing the "people" class
class People:
    def __init__(self, startFloor: int, destinationFloor: int, amount: int = 1):
        self.startFloor = startFloor
        self.destinationFloor = destinationFloor
        self.amount = amount  # Amount of people that's in a group
        self.waitingTimer = 0  # Wait time
        self.journeyTimer = 0  # Journey time
        self.direction = True if (destinationFloor - startFloor > 0) else False
        self.elevatorBreakCount = (
            0  # The amount of pauses before reaching the destination
        )
        self.description = f"[{self.amount} people. Destination: {self.destinationFloor}. Started: {self.startFloor}]"


# Lists of people that are already out. Will not be updated when left elevator
class Outlist:
    def __init__(self, outlist: list = []):
        self.outlist = outlist

    # For adding people into outlist.
    def addOutlist(self, *args: People):
        for i in args:
            self.outlist.append(i)


# Initializing the Elevator class
class Elevator:
    def __init__(
        self,
        direction: bool,
        floor: int,
        minFloor: int,
        maxFloor: int,
        carryLimit: int,
        bufferCutoff: float,
        outList: Outlist,
        progression: float = 0,
        activeSpeed: float = 5,
        bufferSpeed: float = 1,
        *args: People
    ):
        # Properties of elevator
        self.direction = direction  # True -> going up, False = going down
        self.floor = floor
        self.minFloor = minFloor
        self.maxFloor = maxFloor
        self.carryLimit = carryLimit
        self.progression = (
            # In the range 0 - 100 (If enters new floor, progression = 0)
            progression
        )
        self.bufferCutoff = bufferCutoff  # In the range 0 - 100
        self.activeSpeed = activeSpeed
        self.bufferSpeed = bufferSpeed

        # Queues
        self.internalQueue = []
        self.internalQueueNum = len(self.internalQueue)
        self.outList = outList
        for i in args:
            self.peopleIn.append(i)

    # Adding people to external queue
    def addExternalQueue(self, *args: People):
        for i in args:
            self.externalQueue.append(i)

    # Adding people to internal queue
    def addInternalQueue(self, *args: People):
        for i in args:
            self.internalQueue.append(i)
        self.internalQueueNum += len(args)

    # Unload people from internal queue to outlist
    # Checks every people in internal queue. If the destination floor matches the elevator's current floor, move the people to the outlist
    def unloadFromInternalQueue(self):
        # Creating a boolean list. True if the destination floor matches the current floor, false otherwise.
        outQueueBool = [i.destinationFloor == self.floor for i in self.internalQueue]
        internalTemp = []
        # If true, move to outlist, pass otherwise.
        for people, boolean in zip(self.internalQueue, outQueueBool):
            if boolean:
                self.outList.addOutlist(people)
            else:
                internalTemp.append(people)
        self.internalQueue = internalTemp

    # Load people from external queue to internal queue
    # Checks every people in external queue. Pop if possible.
    ### Will be implemented
    def loadFromExternalQueue(self):
        global externalTemp
        # Creating a boolean list. True if the current floor matches the starting floor, false otherwise.
        inQueueBool = [i.startFloor == self.floor for i in externalTemp]
        self.internalQueueNum = len(self.internalQueue)
        freeSpaceNum = self.carryLimit - self.internalQueueNum
        # If true, move to temporary queues
        inQueueTemp = []
        exQueueTemp = []
        for people, boolean in zip(externalTemp, inQueueBool):
            if boolean:
                inQueueTemp.append(people)
            else:
                exQueueTemp.append(people)
        # Finding the cumulative sum of people
        cumulativeSum = 0
        loadableQueueTemp = []
        for people in inQueueTemp:
            cumulativeSum += people.amount
            if cumulativeSum > freeSpaceNum:
                exQueueTemp.append(people)
            else:
                loadableQueueTemp.append(people)
        self.internalQueue.extend(loadableQueueTemp)
        externalTemp = exQueueTemp

    # Progressing the elevator
    def progressElevator(self):
        if self.floor >= self.maxFloor:
            self.direction = not self.direction
            self.floor -= 1
            self.progression = 0
        if self.floor <= self.minFloor:
            self.direction = not self.direction
            self.floor += 1
            self.progression = 0

        if self.progression >= self.bufferCutoff:
            speed = self.bufferSpeed
        else:
            speed = self.activeSpeed

        self.progression += speed
        if self.progression > 100:
            self.progression = 0
            if self.direction == True:
                self.floor += 1
            else:
                self.floor -= 1

    # def directionalScan(self):
    #     internalQueueFiltered = self.internalQueue.sort(key = lambda person: person.amount)
    #     externalQueueFiltered = filter(lambda person: person.direction == self.direction, self.externalQueue)

def parsePeople(peopleList):
    return [i.description for i in peopleList]

In [89]:
# Display functions

def parsePeople(peopleList):
    return [i.description for i in peopleList]

In [101]:
# I forgot how lists in funcitons worked
def doshit(thelist):
    thelist.append('1')
startlist = ['2']
doshit(startlist)
print(startlist)

class doshitter():
    def doshit(self, thelist):
        thelist = [0]

doshitter3000 = doshitter()
doshitter3000.doshit(startlist)
print(startlist)

nextlist = ['a','b','c', 'd', 'e', 'f', 'g']
for letter in nextlist:
    print(f"{nextlist.index(letter)} : {nextlist.pop(nextlist.index(letter))}")


nextlist = ['a','b','c', 'd', 'e', 'f', 'g']
for i, letter in list(enumerate(nextlist)):
    #letter = nextlist.pop(i)
    print(f"{i} : {letter}")

#
#nextlist = [1,2,3]
#for i in nextlist:
#    nextlist.append(i)
#    print(nextlist)

#PYTHON LISTS - I ASSUME PYTHON KEEPS AN INTERNAL INDEX OR POINTER THAT IT USES TO ITERATE THROUGH. IT DOES NOT MOVE IT WHEN ITEMS ARE REMOVED OR ADDED - BEWARE!

['2', '1']
['2', '1']
0 : a
1 : c
2 : e
3 : g
0 : a
1 : b
2 : c
3 : d
4 : e
5 : f
6 : g


In [99]:
testOutlist = Outlist()
testElevator = Elevator(
    direction=False,
    floor=1,
    minFloor=1,
    maxFloor=3,
    carryLimit=10,
    bufferCutoff=60,
    progression=99,
    outList=testOutlist,
)

#Active speed = 5, buffer speed = 1, buffer cutoff at 60
print("Test: Progressing elevator")

print(f"Floor : {testElevator.floor}")
for i in range(12):
    testElevator.progressElevator()
    print(testElevator.progression)
    print(f"Floor : {testElevator.floor}")

print("This should take it right to the cutoff point, ")

Test: Progressing elevator
Floor : 1
5
Floor : 2
10
Floor : 2
15
Floor : 2
20
Floor : 2
25
Floor : 2
30
Floor : 2
35
Floor : 2
40
Floor : 2
45
Floor : 2
50
Floor : 2
55
Floor : 2
60
Floor : 2
This should take it right to the cutoff point, 


In [95]:
#Test:
#Start from floor 1, load people
#Go to floor 2 (hard code, not elevator behavior), unload + get more

testOutlist = Outlist()
testElevator = Elevator(
    direction=False,
    floor=1,
    minFloor=1,
    maxFloor=3,
    carryLimit=10,
    bufferCutoff=60,
    progression=99,
    outList=testOutlist,
)

print("INIT")
externalTemp = [People(1, 2), People(1, 3), People(1, 4), People(2,4)]
print(f"External Queue = {parsePeople(externalTemp)}\n")

print("1: Call LoadFromExternalQueue()")
testElevator.loadFromExternalQueue()

print(f"Internal Queue = {parsePeople(testElevator.internalQueue)}")
print(f"External Queue = {parsePeople(externalTemp)}")

print("2: Call unloadFromInternalQueue()")
testElevator.floor = 2
testElevator.unloadFromInternalQueue()
print(f"Internal Queue = {parsePeople(testElevator.internalQueue)}")
print(f"External Queue = {parsePeople(externalTemp)}")
print(f"Outlist = {parsePeople(testElevator.outList.outlist)}")

#works lel

INIT
External Queue = ['[1 people. Destination: 2. Started: 1]', '[1 people. Destination: 3. Started: 1]', '[1 people. Destination: 4. Started: 1]', '[1 people. Destination: 4. Started: 2]']

1: Call LoadFromExternalQueue()
Internal Queue = ['[1 people. Destination: 2. Started: 1]', '[1 people. Destination: 3. Started: 1]', '[1 people. Destination: 4. Started: 1]']
External Queue = ['[1 people. Destination: 4. Started: 2]']
2: Call unloadFromInternalQueue()
Internal Queue = ['[1 people. Destination: 3. Started: 1]', '[1 people. Destination: 4. Started: 1]']
External Queue = ['[1 people. Destination: 4. Started: 2]']
Outlist = ['[1 people. Destination: 2. Started: 1]']


In [ ]:
#Test:
#Start from floor 1, move up one floor

testOutlist = Outlist()
testElevator = Elevator(
    direction=False,
    floor=1,
    minFloor=1,
    maxFloor=3,
    carryLimit=10,
    bufferCutoff=60,
    progression=99,
    outList=testOutlist,
)

print("INIT")
externalTemp = [People(1, 2), People(1, 3), People(1, 4), People(2,4)]

In [93]:
# INIT

In [94]:
# Main Debug Loop

def mainLoop():
    #Put main loop here
    print("I dida main loop.")

def display():
    #printout current state
    print("printout")

while (True):
    command = input("")
    if (command == "amongus"):
        print("lol")
    elif (command == "exit"):
        break